# `ipywidgets` Interactive Demo

Simple demonstration of rendering Interactive widgets in a `jupyterlite` notebook.

Note that `ipywidgets` is installed by default in Pyolite

In [ ]:
from ipywidgets import IntSlider

In [ ]:
slider = IntSlider()

In [ ]:
slider

In [ ]:
slider

In [ ]:
slider.value

In [ ]:
slider.value = 5

In [ ]:
from ipywidgets import IntText, link

In [ ]:
text = IntText()

In [ ]:
text

In [ ]:
link((slider, 'value'), (text, 'value'));

# `bqplot` Interactive Demo

Plotting in JupyterLite

`bqplot` can be installed in this deployment (it provides the bqplot federated extension), but you will need to make your own deployment to have access to other interactive widgets libraries.

In [ ]:
import micropip

await micropip.install('bqplot')

In [ ]:
from bqplot import *

import numpy as np
import pandas as pd

np.random.seed(0)

n = 100

x = list(range(n))
y = np.cumsum(np.random.randn(n)) + 100.

sc_x = LinearScale()
sc_y = LinearScale()

lines = Lines(
    x=x, y=y,
    scales={'x': sc_x, 'y': sc_y}
)
ax_x = Axis(scale=sc_x, label='Index')
ax_y = Axis(scale=sc_y, orientation='vertical', label='lines')

Figure(marks=[lines], axes=[ax_x, ax_y], title='Lines')

In [ ]:
lines.colors = ['green']

In [ ]:
lines.fill = 'bottom'

In [ ]:
lines.marker = 'circle'

In [ ]:
n = 100

x = list(range(n))
y = np.cumsum(np.random.randn(n))

sc_x = LinearScale()
sc_y = LinearScale()

bars = Bars(
    x=x, y=y,
    scales={'x': sc_x, 'y': sc_y}
)
ax_x = Axis(scale=sc_x, label='Index')
ax_y = Axis(scale=sc_y, orientation='vertical', label='bars')

Figure(marks=[bars], axes=[ax_x, ax_y], title='Bars', animation_duration=1000)

In [ ]:
bars.y = np.cumsum(np.random.randn(n))

# `ipydatagrid` Interactive Demo

Datagrid in JupyterLite

`ipydatagrid` can be installed in this deployment (it provides the ipydatagrid federated extension), but you will need to make your own deployment to have access to other interactive widgets libraries.

In [ ]:
import micropip

await micropip.install('ipydatagrid')

In [ ]:
from random import uniform

import pandas as pd

from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr

from py2vega.constants import SQRT1_2

n_columns = 100
n_rows = 100


data = {"data": [], "schema": {}}

data["data"] = [
    [uniform(0, 1) for c in range(n_columns)] for r in range(n_rows)
]
data["schema"]["fields"] = [
    {"name": str(c), type: "number"} for c in range(n_columns)
]


def renderer_function(cell, default_value):
    return "#fc8403" if cell.value < SQRT1_2 else default_value


conditional_expression = Expr(renderer_function)

default_renderer = TextRenderer(
    background_color=conditional_expression, format=".3f"
)

df = pd.DataFrame(data["data"])

datagrid = DataGrid(df, default_renderer=default_renderer)

In [ ]:
from ipywidgets import FloatSlider, Dropdown, ColorPicker, HBox, VBox

operator_dropdown = Dropdown(options=["<", ">"], value="<")
reference_slider = FloatSlider(value=0.5, min=0, max=1)
output_colorpicker = ColorPicker(value="#fc8403")


def on_change(*args, **kwargs):
    conditional_expression.value = "'{color}' if cell.value {operator} {reference} else default_value".format(
        operator=operator_dropdown.value,
        reference=reference_slider.value,
        color=output_colorpicker.value,
    )


operator_dropdown.observe(on_change, "value")
reference_slider.observe(on_change, "value")
output_colorpicker.observe(on_change, "value")

hbox = HBox((operator_dropdown, reference_slider, output_colorpicker))
VBox([datagrid, hbox])

# `ipygany` Interactive Demo

3D Scientific visualization in JupyterLite

`ipygany` can be installed in this deployment (it provides the ipygany federated extension), but you will need to make your own deployment to have access to other interactive widgets libraries.

In [ ]:
import micropip

await micropip.install('ipygany')

In [ ]:
import numpy as np
from ipywidgets import FloatSlider, FloatRangeSlider, Dropdown, Select, VBox, AppLayout, jslink
from ipygany import Scene, IsoColor, Threshold, PolyMesh, Component, ColorBar, colormaps


# Create triangle indices
nx = 100
ny = 100

triangle_indices = np.empty((ny - 1, nx - 1, 2, 3), dtype=int)

r = np.arange(nx * ny).reshape(ny, nx)

triangle_indices[:, :, 0, 0] = r[:-1, :-1]
triangle_indices[:, :, 1, 0] = r[:-1, 1:]
triangle_indices[:, :, 0, 1] = r[:-1, 1:]

triangle_indices[:, :, 1, 1] = r[1:, 1:]
triangle_indices[:, :, :, 2] = r[1:, :-1, None]

triangle_indices.shape = (-1, 3)

# Create vertices
x = np.arange(-5, 5, 10/nx)
y = np.arange(-5, 5, 10/ny)

xx, yy = np.meshgrid(x, y, sparse=True)

z = np.sin(xx**2 + yy**2) / (xx**2 + yy**2)

vertices = np.empty((ny, nx, 3))
vertices[:, :, 0] = xx
vertices[:, :, 1] = yy
vertices[:, :, 2] = z
vertices = vertices.reshape(nx * ny, 3)

height_component = Component(name='value', array=z)

mesh = PolyMesh(
    vertices=vertices,
    triangle_indices=triangle_indices,
    data={'height': [height_component]}
)

height_min = np.min(z)
height_max = np.max(z)

# Colorize by height
colored_mesh = IsoColor(mesh, input='height', min=height_min, max=height_max)

# Create a slider that will dynamically change the boundaries of the colormap
colormap_slider_range = FloatRangeSlider(value=[height_min, height_max], min=height_min, max=height_max, step=(height_max - height_min) / 100., description='color')

jslink((colored_mesh, 'range'), (colormap_slider_range, 'value'))

# Create a colorbar widget
colorbar = ColorBar(colored_mesh)

# Colormap choice widget
colormap = Dropdown(
    options=colormaps,
    description='colormap:'
)

jslink((colored_mesh, 'colormap'), (colormap, 'index'))

# Threshold effect
threshold_mesh = Threshold(colored_mesh, input='height', min=height_min, max=height_max)

# Create a slider that will dynamically change the boundaries of the threshold
threshold_slider_range = FloatRangeSlider(value=[height_min, height_max], min=height_min, max=height_max, step=(height_max - height_min) / 100., description="threshold")

jslink((threshold_mesh, 'range'), (threshold_slider_range, 'value'))


AppLayout(
    left_sidebar=Scene([threshold_mesh]),
    right_sidebar=VBox((colormap_slider_range, colormap, colorbar, threshold_slider_range)),
    pane_widths=[2, 0, 1]
)